In [115]:
#Import Tools
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import openpyxl

In [116]:
# #retrieve data regarding vacant spaces
from openpyxl import load_workbook
workbook = load_workbook(filename="Node_Information_TU.xlsx")
workbook.sheetnames
sheet1 = workbook.worksheets[0]
sheet2 = workbook.worksheets[1]

# #create dictionary based on excel worksheet
vacant_spaces={}
for value in sheet1.iter_rows(min_row=2, values_only=True):
    for index,item in enumerate(value):
        vacant_spaces[value[0]]={}
        vacant_spaces[value[0]]["location"]=value[1]
        vacant_spaces[value[0]]["tag"]=value[6]
        vacant_spaces[value[0]]["building"]=value[3]
        vacant_spaces[value[0]]["size_real"]=int(value[2])
        vacant_spaces[value[0]]["loc_building"]=value[4]
        vacant_spaces[value[0]]["orientation"]=value[5]
        vacant_spaces[value[0]]["node_type"]=value[7]

#interpret data
#Rules:
#structure (roof:1,ground:3,outside:3,basement:3,intermediate:2)
#solar (roof:3,
#       basement:1
#       ground & S,SW,SE:3,
#       ground $ E,W:2
#       ground $ N,NE,NW:1
#       outside & S,SW,SE:3,
#       outside $ E,W:2
#       outside $ N,NE,NW:1
#       intermediate & S,SW,SE:3,
#       intermediate & E,W:2
#       intermediate & N,NE,NW:1
#size (0-100 size:1)
#     (101-500 size:2)
#     (500-... size:3)

for space in vacant_spaces:
    if vacant_spaces[space]["loc_building"]=="roof":
        vacant_spaces[space]["structure"]=1
        vacant_spaces[space]["solar"]=3
    if vacant_spaces[space]["loc_building"]=="basement":
        vacant_spaces[space]["structure"]=3
        vacant_spaces[space]["solar"]=1
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
        vacant_spaces[space]["structure"]=3
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
        vacant_spaces[space]["structure"]=3
    if (vacant_spaces[space]["loc_building"]=="ground") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
        vacant_spaces[space]["structure"]=3
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
        vacant_spaces[space]["structure"]=2
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
        vacant_spaces[space]["structure"]=2
    if (vacant_spaces[space]["loc_building"]=="intermediate") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
        vacant_spaces[space]["structure"]=2
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="S") or ((vacant_spaces[space]["orientation"])=="SE") or ((vacant_spaces[space]["orientation"])=="SW")):
        vacant_spaces[space]["solar"]=3
        vacant_spaces[space]["structure"]=3
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="N") or ((vacant_spaces[space]["orientation"])=="NE") or ((vacant_spaces[space]["orientation"])=="NW")):
        vacant_spaces[space]["solar"]=1
        vacant_spaces[space]["structure"]=3
    if (vacant_spaces[space]["loc_building"]=="outside") and (((vacant_spaces[space]["orientation"])=="E") or ((vacant_spaces[space]["orientation"])=="W")):
        vacant_spaces[space]["solar"]=2
        vacant_spaces[space]["structure"]=3

#interpret size based on predefined ranges
for space in vacant_spaces:
    if vacant_spaces[space]["size_real"]<=200:
        vacant_spaces[space]["size"]=1
    if (vacant_spaces[space]["size_real"]>200) and (vacant_spaces[space]["size_real"]<=1000) :
        vacant_spaces[space]["size"]=2
    if (vacant_spaces[space]["size_real"]>1000):
        vacant_spaces[space]["size"]=3
     
#retrieve waste sources
#create dictionary based on excel worksheet
wastes={}
for value in sheet2.iter_rows(min_row=2, values_only=True):
    for index,item in enumerate(value):
            wastes[value[0]]={}
            wastes[value[0]]["location"]=value[1]
            wastes[value[0]]["tag"]=value[5]
            wastes[value[0]]["type"]=value[3]
# print(wastes)

In [117]:
workbook = load_workbook(filename="Node_Information_TU.xlsx")
workbook.sheetnames
sheet1 = workbook.worksheets[0]
sheet2 = workbook.worksheets[1]
wastes={}
for value in sheet2.iter_rows(min_row=2, values_only=True):
    for index,item in enumerate(value):
        if value[4]!=0:
            wastes[value[0]]={}
            wastes[value[0]]["location"]=value[1]
            wastes[value[0]]["source"]=value[2]
            wastes[value[0]]["tag"]=value[5]
            wastes[value[0]]["type"]=value[3]
            wastes[value[0]]["quantity"]=int(value[4])
            wastes[value[0]]["node_type"]="waste"


#assign ranges to quantities
for waste in wastes:
    if wastes[waste]["type"]=="W1":
        if wastes[waste]["quantity"]<= 183705:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>183705) and (wastes[waste]["quantity"]<=918523):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>918523:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W2":
        if wastes[waste]["quantity"]<= 67500:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>67500) and (wastes[waste]["quantity"]<=337500):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>337500:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W3":
        if wastes[waste]["quantity"]<= 11813:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>11813) and (wastes[waste]["quantity"]<=59063):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>59063:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W4":
        if wastes[waste]["quantity"]<= 28200:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>28200) and (wastes[waste]["quantity"]<=141000):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>141000:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W5":
        if wastes[waste]["quantity"]<= 19342:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>19342) and (wastes[waste]["quantity"]<=96710):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>96710:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W6":
        if wastes[waste]["quantity"]<= 1560000:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>1560000) and (wastes[waste]["quantity"]<=7800000):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>7800000:
            wastes[waste]["size"]=3
    if wastes[waste]["type"]=="W7":
        if wastes[waste]["quantity"]<= 200000:
            wastes[waste]["size"]=1
        if (wastes[waste]["quantity"]>200000) and (wastes[waste]["quantity"]<=1000000):
            wastes[waste]["size"]=2
        if wastes[waste]["quantity"]>1000000:
            wastes[waste]["size"]=3

In [118]:
#Import Used Waste List
url1="https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/used%20waste.txt"
resp1 = requests.get(url1)
used_waste1 = json.loads(resp1.text)

url1="https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/used%20waste2.txt"
resp1 = requests.get(url1)
used_waste2 = json.loads(resp1.text)

url1="https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/used%20waste3.txt"
resp1 = requests.get(url1)
used_waste3 = json.loads(resp1.text)

url1="https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/used%20waste4.txt"
resp1 = requests.get(url1)
used_waste4 = json.loads(resp1.text)

In [119]:
#Import Space & System List
url = "https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/occupied%20nodes.txt"
resp = requests.get(url)
occupied_nodes1 = json.loads(resp.text)

url = "https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/occupied%20nodes2.txt"
resp = requests.get(url)
occupied_nodes2 = json.loads(resp.text)

url = "https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/occupied%20nodes3.txt"
resp = requests.get(url)
occupied_nodes3 = json.loads(resp.text)

url = "https://raw.githubusercontent.com/erengozdeanil/UF-DecisionMaker/main/occupied%20nodes4.txt"
resp = requests.get(url)
occupied_nodes4 = json.loads(resp.text)

In [120]:
occ_space={}
for space in occupied_nodes1:
    if occupied_nodes1[space]["system"]==None:
        continue
    else:
        occ_space[space]={}
        occ_space[space]["size"]=vacant_spaces[space]["size_real"]
        occ_space[space]["system"]=occupied_nodes1[space]["system"] 

for space in occupied_nodes2:
    if occupied_nodes2[space]["system"]==None:
        continue
    else:
        occ_space[space]={}
        occ_space[space]["size"]=vacant_spaces[space]["size_real"]
        occ_space[space]["system"]=occupied_nodes2[space]["system"] 

for space in occupied_nodes3:
    if occupied_nodes3[space]["system"]==None:
        continue
    else:
        occ_space[space]={}
        occ_space[space]["size"]=vacant_spaces[space]["size_real"]
        occ_space[space]["system"]=occupied_nodes3[space]["system"]

for space in occupied_nodes4:
    if occupied_nodes4[space]["system"]==None:
        continue
    else:
        occ_space[space]={}
        occ_space[space]["size"]=vacant_spaces[space]["size_real"]
        occ_space[space]["system"]=occupied_nodes4[space]["system"]

In [121]:
#Yields per m2
# UF1 : 
uf1_product="worm"
uf1_yield=4.87
# UF2 : 
uf2_product="fish"
uf2_yield=15
# UF3 : 
uf3_product="mushroom"
uf3_yield=125
# UF4 : 
uf4_product="small veg"
uf4_yield=99.9
# UF5 : 
uf5_product="small veg"
uf5_product2="big veg"
uf5_yield=12.5
uf5_yield2=89.6
# UF6 : 
uf6_product="small veg"
uf6_product2="big veg"
uf6_yield=12.5
uf6_yield2=2.7
# UF7 : 
uf7_product="small veg"
uf7_yield=12.5
# UF8 : 
uf8_product="small veg"
uf8_yield=87.4
# UF9 : 
uf9_product="small veg"
uf9_yield=12.5


In [122]:
for space in occ_space:
    occ_space[space]["produce"]=None
    occ_space[space]["produce2"]=None
    occ_space[space]["yield"]=None
    occ_space[space]["yield2"]=None
    if occ_space[space]["system"]=="UF1":
        occ_space[space]["produce"]=uf1_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf1_yield
    if occ_space[space]["system"]=="UF2":
        occ_space[space]["produce"]=uf2_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf2_yield
    if occ_space[space]["system"]=="UF3":
        occ_space[space]["produce"]=uf3_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf3_yield
    if occ_space[space]["system"]=="UF4":
        occ_space[space]["produce"]=uf4_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf4_yield
    if occ_space[space]["system"]=="UF5":
        occ_space[space]["produce"]=uf5_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf5_yield
        occ_space[space]["produce2"]=uf5_product2
        occ_space[space]["yield2"]=occ_space[space]["size"]*uf5_yield2
    if occ_space[space]["system"]=="UF6":
        occ_space[space]["produce"]=uf6_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf6_yield
        occ_space[space]["produce2"]=uf6_product2
        occ_space[space]["yield2"]=occ_space[space]["size"]*uf6_yield2
    if occ_space[space]["system"]=="UF7":
        occ_space[space]["produce"]=uf7_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf7_yield
    if occ_space[space]["system"]=="UF8":
        occ_space[space]["produce"]=uf8_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf8_yield
    if occ_space[space]["system"]=="UF9":
        occ_space[space]["produce"]=uf9_product
        occ_space[space]["yield"]=occ_space[space]["size"]*uf9_yield

In [123]:
yields={}
worm=[]
fish=[]
mushroom=[]
smallveg=[]
bigveg=[]
for space in occ_space:
    yields["worm"]={}
    yields["fish"]={}
    yields["mushroom"]={}
    yields["small veg"]={}
    yields["big veg"]={}
    if occ_space[space]["produce"]=="worm":
        worm.append(occ_space[space]["yield"])
    if occ_space[space]["produce"]=="fish":
        fish.append(occ_space[space]["yield"])
    if occ_space[space]["produce"]=="mushroom":
        mushroom.append(occ_space[space]["yield"])
    if occ_space[space]["produce"]=="small veg":
        smallveg.append(occ_space[space]["yield"])
    if occ_space[space]["produce2"]=="big veg":
        bigveg.append(occ_space[space]["yield"])


In [124]:

worm_kg=sum(worm)
fish_kg=sum(fish)
mushroom_kg=sum(mushroom)
smallveg_kg=sum(smallveg)
bigveg_kg=sum(bigveg)

for product in yields:
    yields[product]["number"]=[]
    yields[product]["weight"]=[]
    if product=="worm":
        yields["worm"]["weight"]=worm_kg
        worm_count=int(worm_kg/0.00025)
        yields["worm"]["number"]=worm_count   
    if product=="fish":
        yields["fish"]["weight"]=fish_kg
        yields["fish"]["number"]=int(fish_kg/(0.25))
    if product=="mushroom":
        yields["mushroom"]["weight"]=mushroom_kg
        yields["mushroom"]["number"]=int(mushroom_kg/(0.15))
    if product=="small veg":
        yields["small veg"]["weight"]=smallveg_kg
        yields["small veg"]["number"]=int(smallveg_kg/(0.14))
    if product=="big veg":
        yields["big veg"]["weight"]=bigveg_kg
        yields["big veg"]["number"]=int(bigveg_kg/(0.015))
print(yields)

{'worm': {'number': 446832239, 'weight': 111708.05999999998}, 'fish': {'number': 483660, 'weight': 120915}, 'mushroom': {'number': 0, 'weight': 0}, 'small veg': {'number': 35639506, 'weight': 4989530.9}, 'big veg': {'number': 114445000, 'weight': 1716675.0}}


In [125]:
waste_quantity={}
waste_quantity["W1"]={}
waste_quantity["W2"]={}
waste_quantity["W3"]={}
waste_quantity["W4"]={}
waste_quantity["W5"]={}
waste_quantity["W6"]={}
waste_quantity["W7"]={}
quantity1=[]
quantity2=[]
quantity3=[]
quantity4=[]
quantity5=[]
quantity6=[]
quantity7=[]
for item in used_waste4:
    if item!=[]:
        if wastes[item]["type"]=="W1":
            quantity1.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W2":
            quantity2.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W3":
            quantity3.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W4":
            quantity4.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W5":
            quantity5.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W6":
            quantity6.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W7":
            quantity7.append(wastes[item]["quantity"])


In [126]:

for item in (waste_quantity):
    print(item)
    waste_quantity[item]["used quantity"]=[]
    waste_quantity[item]["left quantity"]=[]
    if item == "W1":
        waste_quantity["W1"]["used quantity"]=sum(quantity1)
    if item == "W2":
        waste_quantity["W2"]["used quantity"]=sum(quantity2)
    if item == "W3":
        waste_quantity["W3"]["used quantity"]=sum(quantity3)
    if item == "W4":
        waste_quantity["W4"]["used quantity"]=sum(quantity4)
    if item == "W5":
        waste_quantity["W5"]["used quantity"]=sum(quantity5)
    if item == "W6":
        waste_quantity["W6"]["used quantity"]=sum(quantity6)
    if item == "W7":
        waste_quantity["W7"]["used quantity"]=sum(quantity7)

used_waste_quantity=waste_quantity

W1
W2
W3
W4
W5
W6
W7


In [127]:
waste1=[]
waste2=[]
waste3=[]
waste4=[]
waste5=[]
waste6=[]
waste7=[]

for item in wastes:
    if item in used_waste4:
        continue
    else:
        if wastes[item]["type"]=="W1":
            waste1.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W2":
            waste2.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W3":
            waste3.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W4":
            waste4.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W5":
            waste5.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W6":
            waste6.append(wastes[item]["quantity"])
        if wastes[item]["type"]=="W7":
            waste7.append(wastes[item]["quantity"])
for item in wastes:
    if item in used_waste4:
        continue
    else:
        if wastes[item]["type"] == "W1":
            waste_quantity["W1"]["left quantity"]=sum(waste1)
        if wastes[item]["type"] == "W2":
            waste_quantity["W2"]["left quantity"]=sum(waste2)
        if wastes[item]["type"] == "W3":
            waste_quantity["W3"]["left quantity"]=sum(waste3)
        if wastes[item]["type"] == "W4":
            waste_quantity["W4"]["left quantity"]=sum(waste4)
        if wastes[item]["type"] == "W5":
            waste_quantity["W5"]["left quantity"]=sum(waste5)
        if wastes[item]["type"] == "W6":
            waste_quantity["W6"]["left quantity"]=sum(waste6)
        if wastes[item]["type"] == "W7":
            waste_quantity["W7"]["left quantity"]=sum(waste7)
print(waste_quantity)

{'W1': {'used quantity': 4237, 'left quantity': 127305}, 'W2': {'used quantity': 592000, 'left quantity': []}, 'W3': {'used quantity': 812601, 'left quantity': []}, 'W4': {'used quantity': 130200, 'left quantity': 70840}, 'W5': {'used quantity': 1962, 'left quantity': 2017}, 'W6': {'used quantity': 41050300, 'left quantity': 8066500}, 'W7': {'used quantity': 44000000, 'left quantity': 1732320}}


In [128]:
print(occ_space["V13"])
# print(yields)
# print(waste_quantity)

{'size': 3860, 'system': 'UF6', 'produce': 'small veg', 'produce2': 'big veg', 'yield': 48250.0, 'yield2': 10422.0}


In [129]:
# from openpyxl import Workbook
# wb=Workbook()
# sheet1= wb.create_sheet("assigned systems")
# sheet2= wb.create_sheet("yield")
# sheet3= wb.create_sheet("waste use")

# wb.save('TU Breakdown.xlsx')

In [131]:
import pandas as pd
df = pd.DataFrame.from_dict(occ_space, orient='index') # convert dict to dataframe
df.to_csv('TU space system yield.csv') # write dataframe to file

import pandas as pd
df = pd.DataFrame.from_dict(yields, orient='index') # convert dict to dataframe
df.to_csv('TU yields.csv') # write dataframe to file

import pandas as pd
df = pd.DataFrame.from_dict(waste_quantity, orient='index') # convert dict to dataframe
df.to_csv('TU waste use.csv') # write dataframe to file